In [ ]:
import pandas as pd
#carregando a base Iris
df = pd.read_csv('datasets/iris.csv')

In [ ]:
#Vamos fazer a mudança de tipo da coluna alvo usando LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['Species']) #ajusta a função aos dados
df['class'] = le.transform(df['Species']) #aplica a função aos dados e atribui à coluna alvo
df

In [ ]:
#checando o balance das classes 
df['Species'].value_counts()
#(totalmente balanceado)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Id','class','Species']), df['class'], test_size=0.33, random_state=15, stratify=df['class'])
#neste caso o stratify não tem efeito por ser base já balanceada por padrão

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
#Vamos usar uma árvore de decisão simples
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=10) #inicializa o modelo com valores default, ainda não treinado
dt_clf.fit(X_train, y_train) #ajusta o modelo com os dados de treinamento

In [ ]:
#predicao no conjunto de teste
y_pred = dt_clf.predict(X_test) #executa a predição para os dados de teste

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_true=y_test, y_pred=y_pred))

In [ ]:
print(metrics.confusion_matrix(y_test,y_pred))

Relembrando: <img src='img/metrics_theory.png'></img>

* **Acurácia**: Percentual de acertos gerais (tanto positivos quanto negativos) do classificador para os dados amostrados
* **Precisão**: Dentre todas as amostras que o classificador rotulou como positivas, quantas realmente eram positivas (*Quando um classificador é altamente preciso, ele tem maior confiabilidade ao rotular como positiva uma amostra. Quando ele diz que é de uma classe, é porque é :D*)
* **Recall**: O recall ou sensibilidade ou revocação é a capacidade do classificador de encontrar todas as amostras positivas, ou seja, dentre todas as amostras que eram positivas, quantas o classificador conseguiu identificar (*Quando um classificador tem alta sensibilidade, o número de falsos negativos é baixo. Quando ele diz que não é, é porque não é :D*)
* **F1-Score**: A medida F é uma média harmônica ponderada entre precisão e recall, que atinge seu melhor valor em 1 e o pior em 0. Quando se usa F1, se considera a precisão e o recall com a mesma importância.
* **Suporte**: Indica o número de amostras para cada classe testada
* **Especificidade**: Especificidade é a capacidade do classificador de encontrar todas as amostras da classe negativa (complemento da sensibilidade para problemas com 2 classes)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion': ['gini', 'entropy', 'log_loss'], 
              'splitter': ['best', 'random'],
              'max_depth': range(2,6), 
              'max_features': [2,3,4]} 

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, scoring='accuracy', cv=5, n_jobs=-1, verbose = 3) #verbose indica a quantidade de detalhamento das mensagens apresentadas no fit
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_
grid.best_score_

In [ ]:
y_pred_gs = grid.predict(X_test) #ou grid.best_estimator_.predict(X_test)

In [ ]:
print(metrics.classification_report(y_true=y_test, y_pred=y_pred_gs))

In [ ]:

cm = metrics.confusion_matrix(y_test,y_pred_gs)
print(cm)

from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=grid.best_estimator_.classes_)
disp.plot()

In [ ]:
from sklearn import tree
tree.plot_tree(grid.best_estimator_, feature_names=['sepal_L','sepal_W','petal_L','petal_W'],class_names=['setosa','versicolor','virginica'],
               filled=True, rounded=True, fontsize=7)

In [ ]:
from sklearn.model_selection import cross_val_score
#import numpy as np
scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=10, scoring='accuracy') #para comparar modelos
#print(np.mean(scores))#média do CV
#print(np.std(scores)) #crossvalidation
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
#se quiser salvar tudo para análise posterior detalhada
dfgs = pd.DataFrame(grid.cv_results_)

In [ ]:
dfgs.loc[dfgs['rank_test_score'] == 1, :]

In [ ]:
dfgs.to_csv('./resultadogridsearchdt.csv', index=False)

### Vamos usar a ideia do CV para comparar modelos com seus parâmetros default. O que apresentar melhor média de métrica e desvio padrão, escolhemos para gridsearch

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np
def evaluate_model(model, feature_vector, target):
    X_train_limited, X_validation, y_train_limited, y_validation = train_test_split(feature_vector, target, test_size=0.3, random_state=42)
    model.fit(X_train_limited, y_train_limited)
    print("Accuracy full validation set: %0.2f" % model.score(X_validation, y_validation))
    score = cross_val_score(model, feature_vector, target, cv=10, scoring='accuracy', n_jobs=-1)
    print("Mean Accuracy CV: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, X_train, y_train)

In [ ]:
evaluate_model(dt_clf, X_train, y_train)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
cb_model = GradientBoostingClassifier()
evaluate_model(cb_model, X_train, y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
evaluate_model(rf_clf, X_train, y_train)